In [2]:
import pyarrow as pa

In [3]:
import numba, pandas as pd, numpy as np

In [9]:
def pandasHistogram(colName):
    return df[colName].value_counts()

In [4]:
type(getCounts('A'))

pandas.core.series.Series

In [5]:
import numpy as np

import numba
from numba import cuda

@numba.jit(nopython=True)
def compute_bin(x, n, xmin, xmax):
    # special case to mirror NumPy behavior for last bin
    if x == xmax:
        return n - 1 # a_max always in last bin

    # SPEEDTIP: Remove the float64 casts if you don't need to exactly reproduce NumPy
    bin = np.int32(n * (np.float64(x) - np.float64(xmin)) / (np.float64(xmax) - np.float64(xmin)))

    if bin < 0 or bin >= n:
        return None
    else:
        return bin

@cuda.jit
def histogram(x, xmin, xmax, histogram_out):
    nbins = histogram_out.shape[0]
    bin_width = (xmax - xmin) / nbins

    start = cuda.grid(1)
    stride = cuda.gridsize(1)

    for i in range(start, x.shape[0], stride):
        # note that calling a numba.jit function from CUDA automatically
        # compiles an equivalent CUDA device function!
        bin_number = compute_bin(x[i], nbins, xmin, xmax)

        if bin_number >= 0 and bin_number < histogram_out.shape[0]:
            cuda.atomic.add(histogram_out, bin_number, 1)

@cuda.jit
def min_max(x, min_max_array):
    nelements = x.shape[0]

    start = cuda.grid(1)
    stride = cuda.gridsize(1)

    # Array already seeded with starting values appropriate for x's dtype
    # Not a problem if this array has already been updated
    local_min = min_max_array[0]
    local_max = min_max_array[1]

    for i in range(start, x.shape[0], stride):
        element = x[i]
        local_min = min(element, local_min)
        local_max = max(element, local_max)

    # Now combine each thread local min and max
    cuda.atomic.min(min_max_array, 0, local_min)
    cuda.atomic.max(min_max_array, 1, local_max)


def dtype_min_max(dtype):
    '''Get the min and max value for a numeric dtype'''
    if np.issubdtype(dtype, np.integer):
        info = np.iinfo(dtype)
    else:
        info = np.finfo(dtype)
    return info.min, info.max


@numba.jit(nopython=True)
def get_bin_edges(a, nbins, a_min, a_max):
    bin_edges = np.empty((nbins+1,), dtype=np.float64)
    delta = (a_max - a_min) / nbins
    for i in range(bin_edges.shape[0]):
        bin_edges[i] = a_min + i * delta

    bin_edges[-1] = a_max  # Avoid roundoff error on last point
    return bin_edges


def numba_gpu_histogram(a, bins):
    # Move data to GPU so we can do two operations on it
    a_gpu = cuda.to_device(a)

    ### Find min and max value in array
    dtype_min, dtype_max = dtype_min_max(a.dtype)
    # Put them in the array in reverse order so that they will be replaced by the first element in the array
    min_max_array_gpu = cuda.to_device(np.array([dtype_max, dtype_min], dtype=a.dtype))
    min_max[64, 64](a_gpu, min_max_array_gpu)
    a_min, a_max = min_max_array_gpu.copy_to_host()

    # SPEEDTIP: Skip this step if you don't need to reproduce the NumPy histogram edge array
    bin_edges = get_bin_edges(a, bins, a_min, a_max) # Doing this on CPU for now

    ### Bin the data into a histogram 
    histogram_out = cuda.to_device(np.zeros(shape=(bins,), dtype=np.int32))
    histogram[64, 64](a_gpu, a_min, a_max, histogram_out)

    return histogram_out.copy_to_host(), bin_edges

In [17]:
%timeit numba_gpu_histogram(np.asarray(df['B']),1000)

538 ms ± 1.94 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%timeit pandasHistogram('A')

3.98 s ± 130 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Pyarrow stuff

## Convert df to an arrow file that can be saved on disk

In [4]:
df = pd.read_csv('node-server/uploads/uber-dataset',dtype=np.float32)

In [5]:
df.dtypes

sourceid                                    float32
dstid                                       float32
hod                                         float32
mean_travel_time                            float32
standard_deviation_travel_time              float32
geometric_mean_travel_time                  float32
geometric_standard_deviation_travel_time    float32
dtype: object

In [6]:
df.head()

,sourceid,dstid,hod,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time
0,6.0,5.0,17.0,2917.179932,790.280029,2810.979980,1.32
1,3.0,38.0,20.0,339.739990,254.410004,287.540009,1.67
2,2.0,61.0,7.0,1443.359985,374.619995,1397.729980,1.29
3,7.0,25.0,2.0,152.000000,180.399994,95.339996,2.61
4,2.0,79.0,22.0,2320.409912,324.850006,2299.219971,1.14


In [7]:
pa_df = pa.RecordBatch.from_pandas(df)

In [8]:
file = open('node-server/uploads/uber-dataset.arrow', 'wb')

In [9]:
writer = pa.ipc.RecordBatchStreamWriter(file, pa_df.schema)
writer.write_batch(pa_df)
writer.close()
file.close()

## Write function to read arrow file and return the corresponding pandas dataframe 

In [10]:
def readArrowToDF(source):
    reader = pa.RecordBatchStreamReader(source)
    pa_df = reader.read_all()
    return pa_df.to_pandas()

In [11]:
df2 = readArrowToDF("node-server/uploads/uber-dataset.arrow")

In [86]:
df = pd.read_csv('python-scripts/data-100000k.csv')

13.3 s ± 230 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [79]:
type(df2)

pandas.core.frame.DataFrame

In [81]:
%timeit df2['A'].value_counts()

1.75 s ± 56.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
